In [1]:
import sqlite3
import pandas as pd
import pickle
import os

In [2]:
# Copy the read database over. This will now store the results so we will rename it
!cp ../read_db.sqlite3 bio.sqlite3

In [3]:
# Read split data
split_dir = "../experiment_train/split_data"
split_runs = []
splits = set()

for split_fn in sorted(os.listdir(split_dir)):
    if not split_fn.startswith("split_"):
        continue
    filepath = f"{split_dir}/{split_fn}"
    with open(filepath, "rb") as f:
        split_data = pickle.load(f)

    # Collect all runs from train/val/cv sets
    runs = set()
    if split_data.X_train is not None:
        runs.update(split_data.X_train.index)

    # We only did CV, so this isn't needed:
    # if split_data.X_val is not None:
    #     runs.update(split_data.X_val.index)

    if split_data.X_cv is not None:
        runs.update(split_data.X_cv.index)

    for run in runs:
        split_runs.append((split_data.split_id, run))

    splits.add(split_data.split_id)

# Create Conn
conn = sqlite3.connect("bio.sqlite3")
conn.execute("PRAGMA foreign_keys = ON")

# Create splits table
conn.execute("DROP TABLE IF EXISTS splits")
conn.execute(
    """
      CREATE TABLE IF NOT EXISTS splits (
        split_id TEXT PRIMARY KEY
      )
  """
)
conn.executemany("INSERT OR IGNORE INTO splits VALUES (?)", [(i,) for i in splits])

# Create split_runs table
conn.execute("DROP TABLE IF EXISTS split_runs")
conn.execute(
    """
    CREATE TABLE split_runs (
        split_id TEXT NOT NULL,
        run TEXT NOT NULL,
        PRIMARY KEY (split_id, run),
        FOREIGN KEY (run) REFERENCES runs(run)
        FOREIGN KEY (split_id) REFERENCES splits(split_id)

    )
"""
)

conn.executemany("INSERT INTO split_runs VALUES (?, ?)", split_runs)
conn.commit()

# Verify
print("Split -> Run Count:")
for split_id, count in conn.execute(
    "SELECT split_id, COUNT(*) FROM split_runs GROUP BY split_id ORDER BY split_id"
):
    print(f"  Split {split_id}: {count} runs")

conn.close()

Split -> Run Count:
  Split 1: 319 runs
  Split 2: 315 runs
  Split 3: 312 runs
  Split 4: 307 runs
  Split 5: 194 runs


In [4]:
# Load the train results (to add to the database)
train_results = pd.read_csv("../experiment_train/results.csv", low_memory=False)

# Drop misleading (all NaN) column
train_results.drop(columns=["validate_score"], inplace=True)

# Print subset of columns we may be using
print([c for c in train_results.columns if "__" not in c])

print(f"{train_results.shape=}")

train_results.head()

['config_hash', 'split_id', 'pipeline_name', 'cv_mean', 'cv_std', 'cv_fold_0', 'cv_fold_1', 'cv_fold_2', 'cv_fold_3', 'cv_fold_4', 'hp_steps', 'hp_transform_input', 'hp_verbose', 'hp_Preselection', 'hp_Normalization', 'hp_Model']
train_results.shape=(246986, 121)


,config_hash,split_id,pipeline_name,cv_mean,cv_std,cv_fold_0,cv_fold_1,cv_fold_2,cv_fold_3,cv_fold_4,...,hp_Model__monotonic_cst,hp_Model__oob_score,hp_Model__break_ties,hp_Model__cache_size,hp_Model__coef0,hp_Model__decision_function_shape,hp_Model__degree,hp_Model__kernel,hp_Model__probability,hp_Model__shrinking
0,b6d02eaee5c5d68d29f3a33bee03b2433906963c310d57...,3,xgb,0.927425,0.031961,0.959402,0.939103,0.883772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a5693e81ef838fa9459036f1744aa64cc2f078a07671ea...,3,xgb,0.927425,0.031961,0.959402,0.939103,0.883772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50ad34c4b4e96125530928d3aebf8f00f9385225296f1a...,3,xgb,0.927425,0.031961,0.959402,0.939103,0.883772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ab33ab45e7c26dbbdb2d1ceafb5854ebd6ceea52bc2e29...,3,xgb,0.927425,0.031961,0.959402,0.939103,0.883772,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ef2f8d2a83c651e6141f3a0d057885d7bd8fc1c331c772...,3,xgb,0.926816,0.016560,0.943376,0.910256,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Create connection
conn = sqlite3.connect("bio.sqlite3")
conn.execute("PRAGMA foreign_keys = ON")

# Add train results to the database
train_results.to_sql("train_results", conn, if_exists="replace", index=False)

# Create a view that selects the top 5 models per (pipeline_name, split) grouping
conn.execute(
    """
    CREATE VIEW IF NOT EXISTS top_models AS
    SELECT *
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (
                   PARTITION BY pipeline_name, split_id 
                   ORDER BY cv_mean DESC
               ) as rank
        FROM train_results
    )
    WHERE rank <= 5
"""
)
conn.commit()

# Close the Connection
conn.close()